In [1]:
from functools import partial
import numpy as np
import os
import pandas as pd
import scipy

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt


import sys
sys.path.append('../')
import util

save_fig = partial(util.save_fig, save=True)

In [2]:
dataset_file = '/home/drake/workspace/em-2020/data/combined_dataset_all_final.csv'
index_fields = ['Country', 'Year']
data_fields = ['ENI' ,'POP', 'URB', 'AFL' , 'TI', 'CTS' ,'KAOPEN']
prefix = 'template'
start_year = 2000
end_year = 2015
exclude_countries = ['VEN', 'DZA']
rdata_file = '/home/drake/workspace/em-2020/analysis/D3_gmm/template_output.Rdata'

In [3]:
%get dataset_file index_fields data_fields start_year end_year exclude_countries rdata_file
suppressMessages({
library(dplyr)
library(plm)
library(pdynmc)
})
#lag <- plm::lag # dplyr overwrites lag
`%notin%` <- Negate(`%in%`)

data.df <- subset(
    read.csv(dataset_file), select=c(index_fields, data_fields),
    subset=(Country %notin% exclude_countries) 
    & (start_year <= Year) & (Year <= end_year)
)
data.df['AFL2'] = data.df['AFL']^2
head(data.df %>% 
    group_by(Country) %>% 
    summarize(min=min(Year), max=max(Year), .groups='drop'))
data.panel <- pdata.frame(data.df, index = c("Country", "Year"), drop.index = TRUE, row.names=TRUE)

Country,min,max
<fct>,<int>,<int>
ALB,2000,2011
ARG,2000,2014
ARM,2005,2014
AUS,2000,2015
AUT,2000,2015
AZE,2001,2014


In [12]:
model_params <- list(use.mc.diff = TRUE, use.mc.lev = FALSE, use.mc.nonlin = FALSE,
             include.y = TRUE, varname.y = "ENI", lagTerms.y = 2, 
             fur.con = TRUE, fur.con.diff = TRUE, fur.con.lev = FALSE, 
             varname.reg.fur = c("POP", 'AFL', 'AFL2', 'URB', 'CTS', 'TI', 'KAOPEN'), 
             lagTerms.reg.fur = c(0, 0, 0, 0, 0, 0, 0), 
             include.dum = TRUE, dum.diff = TRUE, dum.lev = FALSE, 
             w.mat = "iid.err", std.err = "corrected", estimation = "twostep", opt.meth = "none",
             inst.stata=FALSE)
model_params

$use.mc.diff
[1] TRUE

$use.mc.lev
[1] FALSE

$use.mc.nonlin
[1] FALSE

$include.y
[1] TRUE

$varname.y
[1] "ENI"

$lagTerms.y
[1] 2

$fur.con
[1] TRUE

$fur.con.diff
[1] TRUE

$fur.con.lev
[1] FALSE

$varname.reg.fur
[1] "POP"    "AFL"    "AFL2"   "URB"    "CTS"    "TI"     "KAOPEN"

$lagTerms.reg.fur
[1] 0 0 0 0 0 0 0

$include.dum
[1] TRUE

$dum.diff
[1] TRUE

$dum.lev
[1] FALSE

$w.mat
[1] "iid.err"

$std.err
[1] "corrected"

$estimation
[1] "twostep"

$opt.meth
[1] "none"

$inst.stata
[1] FALSE

In [11]:
base_params <- list(dat = data.df, 
                    varname.i = "Country", varname.t = "Year", varname.dum = "Year")

pdynmc_m0 <- do.call(pdynmc, modifyList(base_params, model_params))
pdynmc_summary <- summary(pdynmc_m0)
pdynmc_summary


Dynamic linear panel estimation (twostep)
Estimation steps: 2

Coefficients:
           Estimate Std.Err.rob z-value.rob Pr(>|z.rob|)    
L1.ENI     0.440228    0.101249       4.348        1e-05 ***
L2.ENI     0.070313    0.049847       1.411      0.15824    
L0.POP    -0.178084    0.207931      -0.856      0.39200    
L0.AFL    -0.265905    0.433525      -0.613      0.53988    
L0.AFL2   -0.018772    0.021711      -0.865      0.38704    
L0.URB     0.270348    0.354961       0.762      0.44606    
L0.CTS     0.041186    0.018732       2.199      0.02788 *  
L0.TI     -0.001997    0.007331      -0.272      0.78562    
L0.KAOPEN  0.002982    0.007617       0.391      0.69580    
2003       0.022383    0.006262       3.574      0.00035 ***
2004       0.027987    0.009310       3.006      0.00265 ** 
2005       0.029499    0.012905       2.286      0.02225 *  
2006       0.040976    0.018350       2.233      0.02555 *  
2007       0.058082    0.024555       2.365      0.01803 *  
2008   

In [5]:
pdynmc_coef <- data.frame(pdynmc_summary$coefficients)
pdynmc_coef$index <- rownames(pdynmc_coef)
pdynmc_tests <- rbind(
    with(pdynmc_summary$hansenj, 
         data.frame(stat=as.numeric(statistic), p.value=p.value, row.names='J-Test (overid restrictions)')),
    with(pdynmc_summary$slopef, 
         data.frame(stat=as.numeric(statistic), p.value=p.value, row.names='F-Statistic (slope coeff)')),
    with(pdynmc_summary$time.dumf, 
         data.frame(stat=as.numeric(statistic), p.value=p.value, row.names='F-Statistic (time dummies)'))
)
pdynmc_coef$index <- rownames(pdynmc_coef)
save(pdynmc_coef, pdynmc_tests, file=rdata_file)

In [6]:
%get pdynmc_coef --from R
pdynmc_coef

,Estimate,Std.Err.rob,z.value.rob,Pr...z.rob..,index
L1.ENI,0.440228,0.101249,4.348,0.00001,L1.ENI
L2.ENI,0.070313,0.049847,1.411,0.15824,L2.ENI
L0.POP,-0.178084,0.207931,-0.856,0.39200,L0.POP
L0.AFL,-0.265905,0.433525,-0.613,0.53988,L0.AFL
L0.AFL2,-0.018772,0.021711,-0.865,0.38704,L0.AFL2
L0.URB,0.270348,0.354961,0.762,0.44606,L0.URB
L0.CTS,0.041186,0.018732,2.199,0.02788,L0.CTS
L0.TI,-0.001997,0.007331,-0.272,0.78562,L0.TI
L0.KAOPEN,0.002982,0.007617,0.391,0.69580,L0.KAOPEN
2003,0.022383,0.006262,3.574,0.00035,2003
